In [1]:
from common import *

import xml.etree.ElementTree as ET
from pymongo import MongoClient
import pymongo

from pymystem3 import Mystem

Slow version of gensim.models.doc2vec is being used


In [2]:
fname = join(join(DATA_FOLDER, 'rawdata'), 'all_docs_raw.txt')

In [3]:
# all_docs = glob(join(DATA_FOLDER, 'rawdata') + '/**/*.xml', recursive=True)
# all_docs = sorted(all_docs)
# with open(fname, mode='wt', encoding='utf-8') as f:
#     f.write('\n'.join(all_docs))

In [4]:
with open(fname, mode='r', encoding='utf-8') as f:
    all_docs = f.read().splitlines()

FileNotFoundError: [Errno 2] No such file or directory: '../data/rawdata/all_docs_raw.txt'

In [ ]:
len(all_docs)

In [ ]:
def get_node(root, name, lang='ru'):
    for node in root.iter(name):
        if 'lang' in node.attrib:
            if node.attrib['lang'].lower() == lang:
                return node
        else:
            return node

In [ ]:
def parse_xml(fname, mstem, lemmatize=True, save_raw=True):
    doc = {}

    with open(fname, 'r', encoding='utf8') as f:
        text = f.read()
        text = text.replace('&deg', 'deg')
    
    try:
        root = ET.fromstring(text)
    except:
        print(fname)
        return
    
    doc.update(root.attrib)

    raw_texts = {}
    
    SDOBI = get_node(root, 'SDOBI')
    if SDOBI is not None:
        node = SDOBI.find('./B500/ru-b540/ru-b542')
        if node is not None:
            raw_texts['name'] = node.text.lower()
        else:
            print(fname)            
    else:
        print('no SDOBI: ' + fn)

    node = get_node(root, 'abstract')
    if node is not None:
        raw_texts['abstract'] = ' '.join(node.itertext()).lower()

    if SDOBI is not None:
        node = SDOBI.find('./B500/B560/ru-b560')
        if node is not None:
            doc['ru-b560'] = ' '.join(node.itertext())

    node = get_node(root, 'description')
    if node is not None:
        raw_texts['description'] = ' '.join(node.itertext()).lower()

    node = get_node(root, 'claims')
    if node is not None:
        raw_texts['claims'] = ' '.join(node.itertext()).lower()        
    
    
    for k,v in raw_texts.items():
        if save_raw:
            doc[k + '_raw'] = v
        if lemmatize:
            doc[k] = ''.join(mstem.lemmatize(v))    
    
    return doc

# Test

In [7]:
afile = '../data/rawdata/2016/RU/A/20161227/2015116494/2015116494.xml'
c1file = '../data/rawdata/2016/RU/C1/20161227/0002605713/0002605713.xml'
c2file = '../data/rawdata/2016/RU/C2/20160127/0002573901/0002573901.xml'
ufile = '../data/rawdata/2016/RU/U1/20161210/0000166582/0000166582.xml'

In [8]:
mstem = Mystem()
doc = parse_xml(c2file, mstem)

Installing mystem to /home/administrator/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz


In [21]:
print(doc['description'])

область техника настоящий изобретение относиться к способ получение поверхностный покрытие на основа поливинилхлорид (пвх), содержать слой износ на основа пвх и слой на основа поливиниловый спирт. предшествующий уровень техника поверхностный покрытие на основа пвх широко применять для декорирование сооружение, дом или транспортный средство. однако поверхностный покрытие на основа пвх обладать недостаток высвобождение летучий органический соединение (лос), некоторый из который токсичный. для уменьшение или предотвращение высвобождение лос быть предлагать замещать летучий соединение нелетучий соединение. однако этот приводить к значительный снижение жесткость поверхностный покрытие, в тот время как жесткость являться важный требование для упругий поверхностный покрытие, особенно напольный покрытие. также быть предлагать применение полиэфирный или полиамидный слой в качество защитный слой или в качество промежуточный слой в многослойный поверхностный покрытие. поливинилспиртовой (пвс) сло

In [9]:
doc

{'abstract': 'изобретение относиться к способ получение поверхностный покрытие, содержать этап обеспечение слой износ на основа пвх (1), по существо, свободный от неорганический наполнитель, нанесение на упоминать слой на основа пвх (1) композиция верхний покрытие, содержать поливиниловый спирт (пвс) и соединение силан, содержать по меньший мера один аминогруппа, нагревание упоминать слой на основа пвх (1) и упоминать композиция верхний покрытие при температура, эквивалентный или превосходить 150°c, для образование верхний покрытие слой износ (2). результат являться получение поверхностный покрытие, сочетать свойство физический устойчивость и свойство химический и газовый барьер. 2 н. и 12 з.п. ф-лы, 4 ил., 6 табла.\n',
 'abstract_raw': 'изобретение относится к способу получения поверхностного покрытия, содержащему этапы обеспечения слоя износа на основе пвх (1), по существу, свободного от неорганических наполнителей, нанесения на упомянутый слой на основе пвх (1) композиции верхнего п

# Fill database with data

In [5]:
client = MongoClient()
db = client.fips

In [6]:
# result = db.patents.create_index([('doc-number', pymongo.ASCENDING),
#                                    ('kind', pymongo.ASCENDING),
#                                    ('country', pymongo.ASCENDING),
#                                    ('date-publ', pymongo.ASCENDING)], unique=True)
# sorted(list(db.patents.index_information()))

In [6]:
def save_to_mongodb(doc_list):
    client = MongoClient()
    db = client.fips
    patents = db.patents
    
    mstem = Mystem()
    
    for fn in doc_list:
        doc = parse_xml(fn, mstem, True, True)
        if doc is not None:
            patents.insert_one(doc)

In [12]:
len(all_docs)//(cpu_count*100)

395

In [ ]:
split_on = len(all_docs)//(cpu_count*100)

for i,sub_docs in enumerate(tqdm(np.array_split(all_docs, split_on))):
    ! rm -f ../data/out
    ! echo {i} >> ../data/out
    
    parallelizer = Parallel(n_jobs=cpu_count)

    # this iterator returns the functions to execute for each task
    tasks_iterator = ( delayed(save_to_mongodb)(list_block) for 
                      list_block in grouper(100, sub_docs) ) 
    result = parallelizer( tasks_iterator )

 21%|██▏       | 84/395 [23:52<54:56, 10.60s/it]  

../data/rawdata/1999/RU/C1/19990220/0002126409/0002126409.xml


 93%|█████████▎| 369/395 [2:40:03<29:43, 68.61s/it]t]

../data/rawdata/2015/Patents/xrfd201503/DOC/RUNWC2/000/000/002/560/425/document.xml


100%|██████████| 395/395 [2:58:26<00:00, 44.67s/it]


In [16]:
# docs = []
# mstem = Mystem()

# for fn in tqdm(all_docs):
#     doc = parse_xml(fn, mstem, False, False)
#     if doc is not None:
#         patents.insert_one(doc)
# #         docs.append(doc)

In [ ]:
# patents.insert_many(docs)

# Analyze data

In [7]:
from bson.son import SON
from pprint import pprint

In [7]:
pipeline = [
     {"$group": {"_id": "$kind", "count": {"$sum": 1}}},
     {"$sort": SON([("count", -1), ("_id", -1)])}
 ]

pprint.pprint(list(db.patents.aggregate(pipeline)))

[{'_id': 'A ', 'count': 438164},
 {'_id': 'C1', 'count': 380857},
 {'_id': 'C2', 'count': 214003},
 {'_id': 'U1', 'count': 167301},
 {'_id': 'A1', 'count': 60184},
 {'_id': 'C ', 'count': 2527},
 {'_id': 'C9', 'count': 1372},
 {'_id': 'A2', 'count': 1028},
 {'_id': 'A3', 'count': 824},
 {'_id': 'U8', 'count': 650},
 {'_id': 'C8', 'count': 13},
 {'_id': 'A4', 'count': 13},
 {'_id': 'A8', 'count': 4}]


In [9]:
db.patents.find({ 'ru-b560': {'$exists': True}}).count()

591782

In [13]:
pipeline = [
    {"$match":{"ru-b560":{'$exists': True}}},
    {"$group": {"_id": "$kind", "count": {"$sum": 1}}},
    {"$sort": SON([("count", -1), ("_id", -1)])}
 ]

pprint.pprint(list(db.patents.aggregate(pipeline)))

[{'_id': 'C1', 'count': 363083},
 {'_id': 'C2', 'count': 213916},
 {'_id': 'A1', 'count': 10647},
 {'_id': 'C ', 'count': 2042},
 {'_id': 'C9', 'count': 1365},
 {'_id': 'A3', 'count': 398},
 {'_id': 'A2', 'count': 305},
 {'_id': 'C8', 'count': 12},
 {'_id': 'A4', 'count': 9},
 {'_id': 'U8', 'count': 4},
 {'_id': 'U1', 'count': 1}]


In [19]:
topn = db.patents.find({'kind': 'C2', 'ru-b560': {'$exists': True}}, 
                       {'ru-b560': 1, "_id": 0}).skip(24300).limit(100)
pprint(list(topn))

[{'ru-b560': 'JP 61-39778, 05.09.1986. RU 2090004 C1, 10.09.1997. US \n'
             '\n'
             '4893353, 09.01.1990. US 4157542, 20.10.1992.'},
 {'ru-b560': 'RU 2000544 С, 07.09.1993. RU 2148796 C1, 10.05.2000. RU 2147731 '
             'C1, 20.04.2000. US 3304786 А, \n'
             '\n'
             '21.02.1967. US 4303978 А, 01.12.1981. US 4675820 А, 23.06.1987.'},
 {'ru-b560': 'ЕР \n'
             '\n'
             '717034, 19.06.1996. WO 9401398, 20.01.1994. ЕР 580968, '
             '02.02.1994. WO 9512575, 11.05.1995. ТРИНУС Ф.П. '
             'Фармако-терапевтический справочник. - Киев: Здоровья, 1989, '
             'с.263.'},
 {'ru-b560': 'ДАНИЛОВ В.М. и др. Вертолет Ми-8МТВ. - М.: Транспорт, 1995, с. '
             '14-20. US 5826824 A, 27.10.1998. US 5467943 A, 21.11.1995. US '
             '4364533 A,\n'
             '\n'
             '\n'
             '\n'
             ' 21.12.1982. US 6027073 A, 22.02.2000. SU 0328678 A, '
             '15.01.1980.'},
 {'ru-b56

In [139]:
for doc in topn:
    splited = doc['ru-b560'].split(',')
    

In [146]:
topn = db.patents.find({}, {'doc-number': 1, "_id": 0})
numbers = []
for doc in topn:
    numbers.append(doc['doc-number'])

In [155]:
len([n for n in numbers if len(n) > 8])

709242

In [160]:
str56 = 'US 5262019 A1, 16.11.93. СПИЦЫН В.И., КУЗИНА А.Ф. Технеций. - М.: Наука, 1981, с.42-53, \n\n120-122. ГОЛЬДАНСКИЙ В.Н. Новые элементы в периодической системе Д.И.Менделеева. - М.: АТОМИЗДАТ, 1964, с.124-126. RU 2029400 C1, 20.02.95. US 5156722 A1, 20.10.92. US 5183541 A1, 02.02.93.'

In [15]:
import re

p = re.compile('(?:RU|SU)\s+\d{1,}\s+(?:C1|C2|U1|A1|C |C9|A2|A3|U8|C8|A4|A8)')

# print(p.findall('sdfsdg    RU    2029400 C1, 20.02.95. US 5156722 A1'))


topn = db.patents.find({'ru-b560': {'$exists': True}}, 
                       {'ru-b560': 1})

regexed = {}
for doc in topn:
    found = p.findall(doc['ru-b560'])
    if len(found):
        regexed[str(doc['_id'])] = found

In [16]:
len(regexed)

328830

In [53]:
top10 = db.patents.find({'doc-number': {'$regex': '^[0]{0,}2111011' }}).skip(0).limit(10)
pprint(list(top10))

[{'_id': ObjectId('5984ba3cb6b1134b1c638529'),
  'abstract': 'использование: в биотехнология и ветеринарный вирусология и \n'
              'касаться вакцина для \n'
              'специфический профилактика респираторный болезнь крупный '
              'рогатый скот, а также другой патология, обусловливать вирус '
              'инфекционный ринотрахеит, парагрипп-3 и вирусный \n'
              'диарея - болезнь \n'
              'слизистый. вакцина "тривак (виэв)" содержать антиген из '
              'аттенуированный вирус инфекционный ринотрахеит штамм '
              '"тк-а(виэв)в-2" с активность 10 4,5  - 10 6,5  \n'
              'тцд 50/мл , \n'
              'штамм вирус вирусный диарея - болезнь слизистая "вк-1(в-1)" n '
              '28 с активность 10 4,0  - 10 6,0  тцд 50/мл , штамм вирус '
              'парагрипп-3 с активность \n'
              '10 5,0  \n'
              '- 10 7,0  тцд 50/мл  и защитный среда. применение вакцина '
              'позволять существенно сн

# ru-b560

# Get w2v

Get sentences for w2v model and tokenize on the fly

In [ ]:
# iter only over 

for sentence in sent_tokenize(text):
    document.append(list(tokenize(sentence, self.stop_words)))
